In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Deep Learning/Tomato leaf disease classification

/content/drive/MyDrive/Deep Learning/Tomato leaf disease classification


In [3]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
from glob import glob
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import matplotlib.pyplot as plt

In [5]:
Image_size = [224, 224]
train_path = '/content/drive/MyDrive/Deep Learning/Tomato leaf disease classification/train'
valid_path = '/content/drive/MyDrive/Deep Learning/Tomato leaf disease classification/val'

In [6]:
inception = InceptionV3(input_shape = Image_size + [3], weights = 'imagenet',include_top =False)

87910968/87910968 [==============================] - 0s 0us/step


In [7]:
for layer in inception.layers:
  layer.trainable=False

In [8]:
folders = glob(train_path+'/*')

In [9]:
x = Flatten()(inception.output)

In [10]:
prediction = Dense(len(folders),activation='softmax')(x)

In [11]:
model = Model(inputs = inception.input, outputs = prediction )

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
train_gen = ImageDataGenerator(horizontal_flip=True,
                               zoom_range=0.2,
                               shear_range=0.2,
                               rescale=1./255)

test_gen = ImageDataGenerator(rescale=1.0/255)

In [15]:
training_set = train_gen.flow_from_directory(train_path, target_size=(224,224),batch_size=64,class_mode='categorical')

Found 10010 images belonging to 10 classes.


In [16]:
test_set = test_gen.flow_from_directory(valid_path, target_size=(224,224),batch_size=64,class_mode='categorical')

Found 1000 images belonging to 10 classes.


In [ ]:
model_history = model.fit_generator(training_set,validation_data = test_set,epochs=10,steps_per_epoch=len(training_set)//10,validation_steps = len(test_set))

<ipython-input-17-d9145ab72d18>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history = model.fit_generator(training_set,validation_data = test_set,epochs=10,steps_per_epoch=len(training_set)//10,validation_steps = len(test_set))


Epoch 1/10
15/15 [==============================] - 578s 39s/step - loss: 17.2631 - accuracy: 0.1958 - val_loss: 9.7812 - val_accuracy: 0.3020
Epoch 2/10
15/15 [==============================] - 258s 17s/step - loss: 4.9493 - accuracy: 0.4865 - val_loss: 2.9071 - val_accuracy: 0.5650
Epoch 3/10


In [ ]:
plt.plot(model_history.history('loss'),label='train_loss')
plt.plot(model_history.history('val_loss'),label='valid_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(model_history.history('accuracy'),label='train_accuracy')
plt.plot(model_history.history('val_accuracy'),label='valid_accuracy')
plt.legend()
plt.show()

In [ ]:
model.save('tomato_disease_classification.h5')

In [ ]:
y_pred = model.predict(test_set)